# Bibliotecas

In [1]:
import sys

sys.path.append('/eos/user/t/thenriqu/Dark_Matter/LGBM_hhdm_analysis/')

import pprint
import json
from pathlib import Path
import pickle

import hepherolib.data as data

from tqdm import tqdm
import tensorflow as tf
from statsmodels.stats.weightstats import DescrStatsW
from tensorflow.keras.models import load_model

from lgbm.controllers_lgb_v2 import LGBLearner, LGBModel

# Disable GPUs
tf.config.set_visible_devices([], 'GPU')

2024-11-22 15:25:00.137918: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 15:25:00.188219: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Configuração

In [2]:
period = 'APV_16'
year_style = 2016
dataset_year = "APV_2016"
basedir = '/eos/user/t/thenriqu/Dark_Matter/Amostras/hhdmAnalysis_deepJet_Regions/datasets'

# Data folder
dataset_name = basedir.split('/')[-2]
data_path = f"./data/{dataset_name}/{dataset_year}"
Path(data_path).mkdir(parents=True, exist_ok=True)
    
# Setup models folders
models_path = f"./models/{dataset_name}/{dataset_year}"
Path(models_path).mkdir(parents=True, exist_ok=True)

# Lendo metadata

In [3]:
with open("/eos/user/t/thenriqu/Dark_Matter/metadata.json", "r") as f:
    metadata = json.load(f)

ST = metadata.get("datasets").get("ST")
TT = metadata.get("datasets").get("TT")
ZZ = metadata.get("datasets").get("ZZ")
WZ = metadata.get("datasets").get("WZ")
DY = metadata.get("datasets").get("DY")
RESIDUAL = metadata.get("datasets").get("RESIDUAL")
DATA = metadata.get("datasets").get("DATA")

# Lendo datasets

In [4]:
variables = ["RegionID", "evtWeight", "MLP_score_torch", "LeadingLep_pt", "LepLep_pt", "LepLep_deltaR", "LepLep_deltaM", "MET_pt", "MET_LepLep_Mt", "MET_LepLep_deltaPhi", "TrailingLep_pt", "MT2LL", "Nbjets"]
ds = data.read_files(basedir, period, mode="normal", features=variables)

data.join_datasets(ds, "ST", ST.get(period), mode="normal")
data.join_datasets(ds, "TT", TT.get(period), mode="normal")
data.join_datasets(ds, "ZZ", ZZ.get(period), mode="normal")
data.join_datasets(ds, "WZ", WZ.get(period), mode="normal")
data.join_datasets(ds, "DYJetsToLL", DY.get(period), mode="normal")
data.join_datasets(ds, "Residual", RESIDUAL.get(period), mode="normal")

# Datasets to be used
used_datasets = [
    *[dt for dt in ds.keys() if dt.startswith("Signal_")],
    "ST",
    "TT",
    "ZZ",
    "WZ",
    "DYJetsToLL",
    "Residual"
]

for dt_name in used_datasets:
    print(dt_name, ds[dt_name].shape)

# Delete every other dataset
datasets_to_delete = [dt_name for dt_name in ds.keys() if dt_name not in used_datasets]
for dt_name in datasets_to_delete:
    del ds[dt_name]


Loading datasets...


100%|██████████| 72/72 [00:13<00:00,  5.18it/s]


Signal_1000_100 (88335, 13)
Signal_1000_200 (88586, 13)
Signal_1000_300 (88409, 13)
Signal_1000_400 (84874, 13)
Signal_1000_600 (84624, 13)
Signal_1000_800 (70928, 13)
Signal_400_100 (59663, 13)
Signal_400_200 (52864, 13)
Signal_500_100 (68983, 13)
Signal_500_200 (67010, 13)
Signal_500_300 (57893, 13)
Signal_600_100 (72558, 13)
Signal_600_200 (73458, 13)
Signal_600_300 (66828, 13)
Signal_600_400 (59176, 13)
Signal_800_100 (84368, 13)
Signal_800_200 (83831, 13)
Signal_800_300 (82955, 13)
Signal_800_400 (80383, 13)
Signal_800_600 (66585, 13)
ST (25940, 13)
TT (659085, 13)
ZZ (475316, 13)
WZ (12610, 13)
DYJetsToLL (1941030, 13)
Residual (251230, 13)


# Modelo metadata

In [5]:
base_model_name = "multi_signal"
features = [
    "LeadingLep_pt",
    "LepLep_deltaM",
    "LepLep_deltaR",
    "LepLep_pt",
    "MET_LepLep_Mt",
    "MET_LepLep_deltaPhi",
    "MET_pt",
    "MT2LL",
    "Nbjets",
    "TrailingLep_pt"
]

# Predict usando LGB

In [6]:
# Load model
lgb_model = LGBModel(model_fpath=f"{models_path}/LGB_{base_model_name}-clf.model")

# Predict each dataset
for dataset_name, dataset in tqdm(ds.items()):
    X_features = dataset[features]
    Y_pred = lgb_model.predict(X_features, features)
    dataset["LGB_score"] = Y_pred

100%|██████████| 26/26 [30:01<00:00, 69.27s/it] 


# Pedrict usando MLP Keras

In [7]:
# Load model
mlp_model = load_model(f"{models_path}/MLP_{base_model_name}-checkpoint.h5")

# Load zscore stats
zscore = json.load(open(f"{data_path}/MLP_{base_model_name}-weighted_stats.json", "r"))

# Predict each dataset
for dataset_name, dataset in tqdm(ds.items()):
    X_features = dataset[features].copy()
    
    # Since the model was trained under processed data, we need to preprocess it to predict
    for feature in features:
        X_features.loc[:, feature] = (X_features[feature] - zscore[feature]["mean"]) / zscore[feature]["std"]

    Y_pred = mlp_model.predict(X_features, batch_size=256)
    dataset["MLP_score_keras"] = Y_pred

  0%|          | 0/26 [00:00<?, ?it/s]

346/346 [==============================] - 1s 1ms/step


  4%|▍         | 1/26 [00:09<03:58,  9.53s/it]

347/347 [==============================] - 0s 1ms/step


  8%|▊         | 2/26 [00:20<04:09, 10.41s/it]

...

 92%|█████████▏| 24/26 [05:01<00:50, 25.16s/it]

7583/7583 [==============================] - 10s 1ms/step


 96%|█████████▌| 25/26 [09:47<01:43, 103.37s/it]

982/982 [==============================] - 1s 1ms/step


100%|██████████| 26/26 [10:14<00:00, 23.64s/it] 


# Salvar predict datasets

Os plots da previsão serão feitos em outro código, pois a previsão do Keras é lenta

In [8]:
with open(f"{data_path}/{base_model_name}-predicted-data.pickle", "wb") as f:
    pickle.dump(ds, f)